## One-time pad: the perfect cipher
The one-time pad belongs to the **polyalphabetic substitution** methods, in which the individual letters (or characters) are each converted (encoded) into other letters (or characters).

<img src="otp.png">


https://en.wikipedia.org/wiki/One-time_pad

In [ ]:
## Convert numbers into letters
[char]65
[char]90

A PowerShell One-Time Pad

In [ ]:
$offset = 65

## Create a ONE-TIME-PAD
[string] $myRandomPad = [char[]] (Get-Random -InputObject (65..90) -Count 26) -join ''
$myRandomPad
$myRandomPad.Length

$letter = 'A'
$letter -match '^[A-Z]$'  ## Any number of times => '^[A-Z]+$'

Lets create a funtion to **decrypt**.

In [ ]:
function enryptLetter {
    param (
        [byte][char]$letter, 
        [string]$otp
    )
    $offset = 65    
    $otp[$letter - $offset]
}
enryptLetter -letter 'A' -otp $myRandomPad

.. and vice versa to **decrypt**: 

In [ ]:
function decryptLetter {
    param (
        [byte][char]$letter, 
        [string]$otp
    )
    $offset = 65            
    [char]($offset + $myRandomPad.IndexOf($letter))
}
decryptLetter -letter 'I' -otp $myRandomPad

## Enrypt/Decrypt
$cipher = enryptLetter -letter 'B' -otp $myRandomPad
decryptLetter -letter $cipher -otp $myRandomPad

Let's create a cmdlet!

In [ ]:
function Convert-OTPMessage {
    [CmdletBinding(DefaultParameterSetName = 'Enrypt')]
    [Alias('crytp')]
    param (        
        ## [ValidatePattern('^[A-Z]$')]  <= Case agnostic        
        [Parameter(ValueFromPipeline, ValueFromPipelineByPropertyName, ParameterSetName = 'Enrypt', Position = 0)]
        [ValidateScript({ $_ -cmatch '^[A-Z]$' })] ## Case sensitive        
        [char[]] $letters,
        
        [Parameter(ValueFromPipeline, ValueFromPipelineByPropertyName, ParameterSetName = 'Decrypt', Position = 0)]
        [ValidateScript({ $_ -cmatch '^[A-Z]$' })]
        [char[]] $encryptedLetters,

        [Parameter(ValueFromPipelineByPropertyName, ParameterSetName = 'Decrypt', Position = 1)]
        [switch] $decrypt,

        [Parameter(ValueFromPipelineByPropertyName, ParameterSetName = 'Decrypt', Position = 2)]
        [string[]]$otp
    )            
    process {
        $offset = 65
        if ($decrypt) {                        
            foreach ($encryptedLetter in [byte[]]$encryptedLetters) {                    
                [PSCustomObject]@{                    
                    'decryptedLetters' = [char] ($otp[0].IndexOf($encryptedLetter) + 65)
                    'encryptedLetters' = [char] $encryptedLetter                    
                    'otp'              = $otp
                }     
            }                        
        }
        else {
            foreach ($letter in [byte[]]$letters) {    
                ## Generating a new random OTP for every letter provided    
                [string] $otp = [char[]] (Get-Random -InputObject (65..90) -Count 26) -join ''
                
                ## Substitue any given letter with the equivalent from the OTP        
                [PSCustomObject]@{
                    'letters'          = [char] $letter
                    'encryptedLetters' = $otp[$letter - $offset]
                    'otp'              = $otp
                }        
            }
        }
    }
}

## A
$message = Convert-OTPMessage -letters 'WEATHERFORECAST'
$message 

## B
#($message | Select-Object -Property 'encryptedLetters', 'otp' | 
#  Convert-OTPMessage -decrypt).decryptedLetters -join ''

## C
# crytp PSCONFEU  | crytp -decrypt